In [16]:
import pandas as pd
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import re
import os
import shutil
import sddk
import pickle
import json
import nltk
import pickle

In [17]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,tokens_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,630,8839,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6527,82445,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,8905,170205,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,17961,193272,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3533,47878,[Meteorology/Earth sciences],exact_sciences


In [21]:
filtered_vocab_df = pd.read_json("../data/filtered_vocab_df.json")
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared,transl
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034,0.002921,True,"say, call, tell"
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314,0.002680,True,"book, volume, inner bark of a tree, book, book"
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110,0.002563,True,"be able, can"
6100,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,...,0.001686,0.002307,0.002209,0.002487,0.001690,0.001891,0.004597,0.002367,True,"do, make, handle"
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807,0.002310,True,"have, hold, possess, consider, think"
1764,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,...,0.001680,0.002693,0.002146,0.002476,0.001962,0.001895,0.002397,0.002279,True,part
8953,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,...,0.001686,0.001846,0.002739,0.002701,0.002085,0.002239,0.002556,0.002129,True,suddenly
1449,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,...,0.001883,0.001437,0.001897,0.002244,0.001466,0.001554,0.001730,0.001717,True,
2323,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,...,0.001626,0.001578,0.001717,0.001703,0.002230,0.001240,0.001090,0.001632,True,"place, location"
56,aqua,11747,41231,20765,34038,42778,18714,30884,14948,28253,...,0.001232,0.000754,0.001905,0.001437,0.001160,0.001966,0.005758,0.001575,True,"get/fetch/bring water, be watered"


In [22]:
filtered_vocab_df[filtered_vocab_df["word"].str.startswith("alch")]

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared,transl
4213,alchemia,7,354,281,72,85,15,15,15,16,...,5.982000e-07,7.570000e-07,1.078600e-06,9.177000e-07,1.061800e-06,1.039912e-04,0.000063,1.560640e-05,False,alchemy
7158,alchimia,11,31,49,187,66,33,36,20,51,...,1.435800e-06,1.009300e-06,3.438200e-06,1.223600e-06,2.689800e-06,2.991970e-05,0.000113,1.286430e-05,True,alchemy
4534,alchymia,16,70,86,272,200,51,123,34,161,...,4.905500e-06,1.715900e-06,1.085390e-05,3.609700e-06,9.839100e-06,1.358520e-05,0.000034,7.997100e-06,True,
4661,alchymista,0,35,33,162,33,15,127,10,149,...,5.065000e-06,5.047000e-07,1.004490e-05,1.774200e-06,8.918900e-06,6.630900e-06,0.000016,4.548100e-06,False,
5414,alchimista,4,13,5,49,7,10,16,0,41,...,6.381000e-07,0.000000e+00,2.764000e-06,2.447000e-07,1.132600e-06,2.102500e-06,0.000033,3.334500e-06,False,alchemist
10538,alchymicus,4,19,5,34,10,6,34,4,40,...,1.356000e-06,2.019000e-07,2.696600e-06,6.730000e-07,2.123500e-06,2.425900e-06,0.000011,1.762000e-06,False,
6787,alchimicus,0,9,3,20,5,3,2,0,2,...,7.980000e-08,0.000000e+00,1.348000e-07,0.000000e+00,1.416000e-07,3.558000e-06,0.000017,1.728400e-06,False,
5627,alchimias,0,5,1,61,3,0,3,0,9,...,1.196000e-07,0.000000e+00,6.067000e-07,0.000000e+00,2.124000e-07,8.895000e-06,0.000010,1.698400e-06,False,alchemy
8451,alchimius,0,4,1,2,0,2,0,0,1,...,0.000000e+00,0.000000e+00,6.740000e-08,0.000000e+00,0.000000e+00,4.852000e-07,0.000010,8.144000e-07,False,


In [23]:
filtered_vocab_df = filtered_vocab_df[filtered_vocab_df["word"].apply(lambda x: len(x) > 2)]

In [24]:
vocab_freqs = filtered_vocab_df.set_index("word")["1601-1650"].to_dict()

# FastText - development and testing

In [25]:
metadata_table_long["id"].tolist()[0]

705665

In [4]:
source_path = "/srv/data/tome/noscemus/sents_data_jsons/"
id = metadata_table_long["id"].tolist()[0]
f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
f_sents_data[10:15]

[['705665',
  10,
  'Adfulgeant Omnia Bona!',
  [['Adfulgeant', 'adfulgeo', 'VERB', [0, 10]],
   ['Omnia', 'omnis', 'DET', [11, 16]],
   ['Bona', 'Bonum', 'NOUN', [17, 21]],
   ['!', '!', 'PUNCT', [21, 22]]]],
 ['705665',
  11,
  'Digitized by Google Diuina Majestate * Signati Augustissime Augustissima Regum, Regina, Cogitatione, Consiliis, Affectu, Parentes Patriae!',
  [['Digitized', 'digitized', 'VERB', [0, 9]],
   ['by', 'by', 'SCONJ', [10, 12]],
   ['Google', 'Google', 'PROPN', [13, 19]],
   ['Diuina', 'diuinus', 'ADJ', [20, 26]],
   ['Majestate', 'maiestas', 'NOUN', [27, 36]],
   ['*', '*', 'PUNCT', [37, 38]],
   ['Signati', 'signatus', 'VERB', [39, 46]],
   ['Augustissime', 'augustissimus', 'ADV', [47, 59]],
   ['Augustissima', 'augustissimus', 'ADJ', [60, 72]],
   ['Regum', 'rex', 'NOUN', [73, 78]],
   [',', ',', 'PUNCT', [78, 79]],
   ['Regina', 'regina', 'NOUN', [80, 86]],
   [',', ',', 'PUNCT', [86, 87]],
   ['Cogitatione', 'cogitatio', 'NOUN', [88, 99]],
   [',', ',', 'PUNC

In [26]:
# a genrator for iteration without flooding the CPU
class SentsCorpus:
    def __iter__(self):
        source_path = "/srv/data/tome/noscemus/sents_data_jsons/"
        for id in ids:
            f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
            sents_n = len(f_sents_data)
            tokens_n = 0
            lemmata = []
            for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
                tokens_n += len(sent_data)
                lemmasent = []
                for wordform, lemma, tag, position in sent_data:
                    if tag in ["NOUN", "PROPN", "ADJ", "VERB"]:
                        lemmasent.append(lemma)
                lemmasent = [re.sub(r"\W*|\d*", "", t) for t in lemmasent]
                lemmasent = [l.lower() for l in lemmasent if len(l) > 2]
                try:
                    yield lemmasent
                except:
                    pass

In [27]:
# test with a small corpus from one decade
ids = metadata_table_long[metadata_table_long["file_year"].between(1601, 1610)]["id"]
vocab_freqs = filtered_vocab_df[filtered_vocab_df["1601-1650"]>= 10].set_index("word")["1601-1650"].to_dict()
corpus = SentsCorpus()

In [11]:
len(vocab_freqs)

7018

In [19]:
# how many sentences are there in the small corpus?
len([s for s in corpus])

194843

In [12]:
%%time
# let's train the model
model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=8, min_n=5)
model.build_vocab_from_freq(word_freq=vocab_freqs)
model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)

CPU times: user 4min 20s, sys: 4.12 s, total: 4min 24s
Wall time: 1min 26s


(11141390, 16204680)

In [13]:
# most similar words to harmonia
model.wv.most_similar("harmonia")

[('concentus', 0.6159231662750244),
 ('localis', 0.5688068270683289),
 ('duplicao', 0.5572189092636108),
 ('mistura', 0.5461449027061462),
 ('uocalis', 0.5439537167549133),
 ('duplico', 0.539736270904541),
 ('concessio', 0.530397891998291),
 ('concessum', 0.5274697542190552),
 ('parsimonia', 0.5202438831329346),
 ('musicus', 0.5127111077308655)]

# Training test - discipline-based models

In [14]:
nltk.FreqDist([d for dislist in metadata_table_long[metadata_table_long["file_year"].between(1501,1700)]["disciplines_list"] for d in dislist]).most_common()

[('Medicine', 221),
 ('Astronomy/Astrology/Cosmography', 193),
 ('Biology', 141),
 ('Mathematics', 138),
 ('Meteorology/Earth sciences', 127),
 ('Physics', 105),
 ('Geography/Cartography', 84),
 ('Other (see description)', 79),
 ('Alchemy/Chemistry', 52)]

In [28]:
disciplines_list = ['Medicine',
 'Astronomy/Astrology/Cosmography',
 'Biology',
 'Mathematics',
 'Meteorology/Earth sciences',
 'Physics',
 'Geography/Cartography',
 'Alchemy/Chemistry']

In [29]:
total_vocabulary = list(filtered_vocab_df["word"])

In [30]:
discipline = "Biology"
ids = metadata_table_long[(metadata_table_long["file_year"].between(1501,1700)) & (metadata_table_long["disciplines_list"].apply(lambda x: discipline in x))]["id"]
corpus = SentsCorpus()
vocab_freqs = filtered_vocab_df[filtered_vocab_df[discipline]>= 10].set_index("word")[discipline].to_dict()

In [31]:
len(vocab_freqs)

10188

In [22]:
model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=32, min_n=5)
model.build_vocab_from_freq(word_freq=vocab_freqs)
model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)

(107746797, 165105570)

In [27]:
model.wv.most_similar("mercurius")

[('sulphur', 0.661625862121582),
 ('saturnus', 0.657781183719635),
 ('chymistus', 0.6394349932670593),
 ('chymista', 0.5985708832740784),
 ('mars', 0.588187575340271),
 ('argentum', 0.580955445766449),
 ('figo', 0.5711258053779602),
 ('metallum', 0.5707212686538696),
 ('elixir', 0.5687674880027771),
 ('sal', 0.5623406767845154)]

# Main training

In [32]:
version = "v0.5"
vectors_folder = "../data/large_files/vectors_{}/".format(version)
os.makedirs(vectors_folder, exist_ok=True)

## NOSCEMUS disciplines models

In [33]:
%%time
# main training of the models
for discipline in disciplines_list:
    ids = metadata_table_long[(metadata_table_long["file_year"].between(1501,1700)) & (metadata_table_long["disciplines_list"].apply(lambda x: discipline in x))]["id"]
    corpus = SentsCorpus()
    vocab_freqs = filtered_vocab_df[filtered_vocab_df[discipline]>= 10].set_index("word")[discipline].to_dict()
    model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=32, min_n=5)
    model.build_vocab_from_freq(word_freq=vocab_freqs)
    model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)
    discicipline_str = discipline.replace("/", "_").replace(" ", "_")
    print("model for the period {} successfully trained.".format(discicipline_str))
    model.wv.save(vectors_folder + "vectors_{}.wv".format(discicipline_str))

model for the period Biology successfully trained.
model for the period Mathematics successfully trained.
model for the period Meteorology_Earth_sciences successfully trained.
model for the period Physics successfully trained.
model for the period Geography_Cartography successfully trained.
model for the period Alchemy_Chemistry successfully trained.
CPU times: user 5h 31min 40s, sys: 2min 55s, total: 5h 34min 35s
Wall time: 1h 33min 25s


## NOSCEMUS temporal models

In [35]:
periods = [(1501, 1550),
                      (1551, 1600),
                      (1601, 1650),
                      (1651, 1700)]
periods_str = filtered_vocab_df.columns[1:5]
periods_str

Index(['1501-1550', '1551-1600', '1601-1650', '1651-1700'], dtype='object')

In [36]:
%%time
# main training of the models
for per_tup, per_str in zip(periods, periods_str):
    ids = metadata_table_long[metadata_table_long["file_year"].between(per_tup[0], per_tup[1])]["id"]
    vocab_freqs = filtered_vocab_df[filtered_vocab_df[per_str]>= 10].set_index("word")[per_str].to_dict()
    corpus = SentsCorpus()
    model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=32, min_n=5)
    model.build_vocab_from_freq(word_freq=vocab_freqs)
    model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)
    print("model for the period {} successfully trained.".format(per_str))
    model.wv.save(vectors_folder + "vectors_{}.wv".format(per_str))

model for the period 1501-1550 successfully trained.
model for the period 1551-1600 successfully trained.
model for the period 1601-1650 successfully trained.
model for the period 1651-1700 successfully trained.
CPU times: user 3h 12min 1s, sys: 1min 31s, total: 3h 13min 32s
Wall time: 52min 55s


## EMLAP model

In [44]:
emlap_metadata = pd.read_csv("/srv/data/tome/tome-corpus/emlap_metadata.csv", sep=";")
ids = emlap_metadata["No."]

In [45]:

# a genrator for iteration without flooding the CPU
class SentsCorpus:
    def __iter__(self):
        source_path = "/srv/data/tome/tome-corpus/sents_data_id_jsons_v3-0/"
        for id in ids:
            f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
            sents_n = len(f_sents_data)
            tokens_n = 0
            lemmata = []
            for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
                tokens_n += len(sent_data)
                lemmasent = []
                for wordform, lemma, tag, position, t_pages, t_textblocks in sent_data:
                    if tag in ["NOUN", "PROPN", "ADJ", "VERB"]:
                        lemmasent.append(lemma)
                lemmasent = [re.sub(r"\W*|\d*", "", t) for t in lemmasent]
                lemmasent = [l.lower() for l in lemmasent if len(l) > 2]
                try:
                    yield lemmasent
                except:
                    pass

In [46]:
# test with a small corpus from one decade

ids = emlap_metadata["No."]
vocab_freqs = filtered_vocab_df[filtered_vocab_df["EMLAP"]>= 10].set_index("word")["EMLAP"].to_dict()
corpus = SentsCorpus()

In [47]:
len(vocab_freqs)

7145

In [48]:
%%time
# let's train the model
model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=8, min_n=5)
model.build_vocab_from_freq(word_freq=vocab_freqs)
model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)
model.wv.save(vectors_folder + "/vectors_{}.wv".format("EMLAP"))

CPU times: user 8min 53s, sys: 2.84 s, total: 8min 56s
Wall time: 2min 23s


In [40]:
# if you want to upload the vectors to sciencedata:
#shutil.make_archive("../data/large_data/vectors", 'zip', "../data/large_data/vectors") #
#s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/vectors.zip", data=open("../data/large_data/vectors.zip", "rb"))
# shutil.make_archive("../data/large_data/lila_vectors", 'zip', "../data/large_data/lila_vectors") #
# s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/lila_vectors.zip", data=open("../data/large_data/lila_vectors.zip", "rb"))

In [51]:
sorted([f for f in os.listdir("../data/large_files/vectors_v0.5/") if f.endswith(".wv")])

['vectors_1501-1550.wv',
 'vectors_1551-1600.wv',
 'vectors_1601-1650.wv',
 'vectors_1651-1700.wv',
 'vectors_Alchemy_Chemistry.wv',
 'vectors_Astronomy_Astrology_Cosmography.wv',
 'vectors_Biology.wv',
 'vectors_EMLAP.wv',
 'vectors_Geography_Cartography.wv',
 'vectors_Mathematics.wv',
 'vectors_Medicine.wv',
 'vectors_Meteorology_Earth_sciences.wv',
 'vectors_Physics.wv']

In [52]:
labels_vectors_tups = [('NOSCEMUS - 1501-1550', 'vectors_1501-1550.wv'),
 ('NOSCEMUS - 1551-1600', 'vectors_1551-1600.wv'),
 ('NOSCEMUS - 1601-1650', 'vectors_1601-1650.wv'),
 ('NOSCEMUS - 1651-1700', 'vectors_1651-1700.wv'),
 ('NOSCEMUS - Alchemy/Chemistry', 'vectors_Alchemy_Chemistry.wv'),
 ('NOSCEMUS - Astronomy/Astrology/Cosmography',
  'vectors_Astronomy_Astrology_Cosmography.wv'),
 ('NOSCEMUS - Biology', 'vectors_Biology.wv'),
 ('NOSCEMUS - Geography/Cartography', 'vectors_Geography_Cartography.wv'),
 ('NOSCEMUS - Mathematics', 'vectors_Mathematics.wv'),
 ('NOSCEMUS - Medicine', 'vectors_Medicine.wv'),
 ('NOSCEMUS - Meteorology/Earth sciences',
  'vectors_Meteorology_Earth_sciences.wv'),
 ('NOSCEMUS - Physics', 'vectors_Physics.wv'),
                       ("EMLAP", "vectors_EMLAP.wv")
]


In [53]:
vectors_dict_comp = {}
for tup in labels_vectors_tups:
    model = KeyedVectors.load(vectors_folder + tup[1])
    word_vectors = model.vectors
    vocabulary = model.index_to_key
    kv = KeyedVectors(vector_size=word_vectors.shape[1])
    # Fill the keyed vectors
    for word, vector in zip(vocabulary, word_vectors):
        kv.add_vector(word, vector)
    vectors_dict_comp[tup[0]] = kv

In [54]:
# small test
vectors_dict_comp["NOSCEMUS - Biology"].vectors.shape

(10188, 100)

In [55]:
vectors_dict_comp["EMLAP"].vectors.shape

(7145, 100)

In [56]:
vectors_dict_comp["EMLAP"]["mercurius"]


array([-1.26134321e-01, -1.68080851e-01,  1.67357191e-01, -1.71556875e-01,
        2.60638505e-01, -1.82487279e-01,  2.45628580e-01,  1.61375746e-01,
       -4.17936176e-01,  2.25963980e-01, -1.86831281e-01,  1.99185297e-01,
       -2.04117388e-01,  3.02328616e-01, -9.40453559e-02, -3.60448025e-02,
       -3.72127444e-01, -3.39083560e-02,  3.19622830e-02, -2.66010582e-01,
       -3.25596184e-01,  3.08051109e-01, -1.92536861e-01, -6.08736575e-01,
       -4.72155213e-02, -2.57130653e-01,  2.42697164e-01,  1.14430241e-01,
        4.89463717e-01, -1.56421185e-01,  3.08530539e-01, -5.12138188e-01,
       -1.56960651e-01,  4.26146120e-01, -2.55708605e-01,  5.22660673e-01,
       -1.37785188e-04,  5.30817695e-02,  1.61143512e-01,  7.94618502e-02,
       -4.35531978e-03, -1.28337055e-01, -4.13486630e-01, -5.14758348e-01,
        9.00570080e-02,  1.95274185e-02,  8.77671912e-02,  1.71957001e-01,
       -4.79615748e-01,  3.59725416e-01, -3.04212958e-01,  4.64788638e-02,
       -9.07079354e-02,  

## Adding LiLa vectors

In [57]:
for label, model_path in [
        ("LASLA", "/srv/data/lila/allLASLA-lemmi-fast-100-SKIP-win5-min5.vec"),
        ("Opera Maiora" , "/srv/data/lila/opera-maiora-lemmas_skip_100.vec")]:
    model = KeyedVectors.load_word2vec_format(model_path, binary=False)
    word_vectors = model.vectors
    vocabulary = model.index_to_key
    kv = KeyedVectors(vector_size=word_vectors.shape[1])
    # Fill the keyed vectors
    for word, vector in zip(vocabulary, word_vectors):
        kv.add_vector(word, vector)
    vectors_dict_comp[label] = kv


## Saving for future usage

In [58]:
with open("../data/vectors_dict_comp.pkl", "wb") as file:
    pickle.dump(vectors_dict_comp, file)

In [59]:
with open("/srv/data/tome/noscemus/vectors/vectors_dict_comp.pkl", "wb") as file:
    pickle.dump(vectors_dict_comp, file)

In [60]:
with open("/srv/webserver/apps/iweems_app/data/vectors_dict_comp.pkl", "wb") as file:
    pickle.dump(vectors_dict_comp, file)

with open("/home/jupyter-vojta/notebooks/iweems_huggingface/data/vectors_dict_comp.pkl", "wb") as file:
    pickle.dump(vectors_dict_comp, file)
